#### Model Evaluation and Refinement

In [ ]:
# Standard imports
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
sns.set()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DA0101EN-SkillsNetwork/labs/Data%20files/module_5_auto.csv', header=0)

In [ ]:
df.head()

First, let's only use numeric data:

In [ ]:
df_numeric = df._get_numeric_data()

In [ ]:
df_numeric.head()

In [ ]:
df_numeric.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1, inplace=True)

In [ ]:
df_numeric.head()

Libraries for plotting:

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual

Functions for Plotting

In [ ]:
def DistributionPlot(RedFunction, BlueFunction, RedName, BlueName, Title):
    width = 12
    height = 10
    plt.figure(figsize=(width, height))

    ax1 = sns.distplot(RedFunction, hist=False, color="r", label=RedName)
    ax2 = sns.distplot(BlueFunction, hist=False,
                       color="b", label=BlueName, ax=ax1)

    plt.title(Title)
    plt.xlabel('Price (in dollars)')
    plt.ylabel('Proportion of Cars')

    plt.show()
    plt.close()


def PollyPlot(xtrain, xtest, y_train, y_test, lr, poly_transform):
    width = 12
    height = 10
    plt.figure(figsize=(width, height))

    # training data
    # testing data
    # lr:  linear regression object
    # poly_transform:  polynomial transformation object

    xmax = max([xtrain.values.max(), xtest.values.max()])

    xmin = min([xtrain.values.min(), xtest.values.min()])

    x = np.arange(xmin, xmax, 0.1)

    plt.plot(xtrain, y_train, 'ro', label='Training Data')
    plt.plot(xtest, y_test, 'go', label='Test Data')
    plt.plot(x, lr.predict(poly_transform.fit_transform(
        x.reshape(-1, 1))), label='Predicted Function')
    plt.ylim([-10000, 60000])
    plt.ylabel('Price')
    plt.legend()

##### Part 1: Training and Testing

In [ ]:
y_data = df_numeric['price']

In [ ]:
x_data = df_numeric.drop('price', axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.10, random_state=1)

print("number of test samples :", x_test.shape[0])
print("number of training samples:", x_train.shape[0])

In [ ]:
x_train1, x_test1, y_train1, y_test1 = train_test_split(
    x_data, y_data, test_size=0.40, random_state=0)

print("number of test samples :", x_test1.shape[0])
print("number of training samples:", x_train1.shape[0])

In [ ]:
from sklearn.linear_model import LinearRegression
lre = LinearRegression()

In [ ]:
lre.fit(x_train[['horsepower']], y_train)

In [ ]:
lre.score(x_test[['horsepower']], y_test)

In [ ]:
lre.score(x_train[['horsepower']], y_train)

In [ ]:
lre.score(x_test1[['horsepower']], y_test1)

In [ ]:
lre.score(x_train1[['horsepower']], y_train1)

##### Cross-Validation Score
Sometimes you don't have enough data for testing.\
As a result, you may want to perform cross-validaion

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
Rcross = cross_val_score(lre, x_data[['horsepower']], y_data, cv=4)
Rcross

In [ ]:
Rcross.mean()

In [ ]:
Rcross.std()

We can use negative squared error as a score by setting the parameter 'scoring' metric to 'neg_mean_squared_error'

In [ ]:
-1 * cross_val_score(lre,  x_data[['horsepower']],
                     y_data, cv=4, scoring='neg_mean_squared_error')

In [ ]:
Rc = cross_val_score(lre, x_data[['horsepower']], y_data, cv=2)
Rc.mean()

You can also use the function 'cross_val_predict' to predict the output. The \
function splits up the data into the specified number of folds, with one \
fold for testing and the other folds are used for training. First, import the function:

In [ ]:
from sklearn.model_selection import cross_val_predict

In [ ]:
yhat = cross_val_predict(lre, x_data[['horsepower']], y_data, cv=4)

In [ ]:
yhat[0:5]

##### Part 2: Overfitting, Underfitting and Model Selection

Let's create Multiple Linear Regression objects and train the model using 'horsepower', 'curb-weight', 'engine-size' and 'highway-mpg' as features.

In [ ]:
lr = LinearRegression()
lr.fit(x_train[['horsepower', 'curb-weight',
       'engine-size', 'highway-mpg']], y_train)

In [ ]:
yhat_train = lr.predict(x_train[['horsepower', 'curb-weight',
                                 'engine-size', 'highway-mpg']])
yhat_train[0:5]

In [ ]:
yhat_test = lr.predict(x_test[['horsepower', 'curb-weight',
                               'engine-size', 'highway-mpg']])
yhat_test[0:5]

We can perform some model evaluation using the training and testing data separately\
But 1st let's do some plotting

In [ ]:
Title = 'Distribution  Plot of  Predicted Value Using Training Data vs Training Data Distribution'
DistributionPlot(y_train, yhat_train, "Actual Values (Train)",
                 "Predicted Values (Train)", Title)